## List DSC Courses
Use to update [Canvas_URLs](https://docs.google.com/spreadsheets/d/1wmcnDNxnKDVYp4x4J5jUT1ddIFrE2GguX87-pZtHAmk/edit?usp=drive_web&ouid=115225585177953782447) google sheet

In [1]:
#==========================================================================#
# list-dsc-courses
#==========================================================================#
# * List all courses in DSC and Academic Transformation sub accounts  
#
# Cathy Leahy
# Last update: 11/05/2018
#--------------------------------------------------------------------------#
# Setup
# [1] config
# [2] report folder
#
# Output file fields
#   sis_course_id, course_code, 
#   account_id (sub account), id (canvas course id), 
#   workflow_state (unpublised|available), url (canvas course url), 
#   default_view (wiki, module ...)
#--------------------------------------------------------------------------#
# TO DO 
# * improve error handeling
#--------------------------------------------------------------------------#
import csv, requests, time
from time import gmtime, strftime
from pprint import pprint
import json
import pandas as pd


#==========================================================================#
# Variables
#==========================================================================#
# Adapted from G. Q. Maguire Jr.
with open('../config/config.json') as json_data_file:
    configuration = json.load(json_data_file)
    access_token=configuration["canvas"]["access_token"]

header = {'Authorization' : 'Bearer ' + access_token}
payload = {}
baseURL="https://"+configuration["canvas"]["host"]+"/api/v1/"
log_file = 'log.txt' # timestamped log file

#==========================================================================#
# Main
#==========================================================================#
def main():
    printDate = strftime("%Y-%m-%d", time.localtime())  
    
    # DSC sub account (700)      
    print_log('DSC   700')  
    account_id = 700
    url='{}accounts/{}/courses?per_page=100'.format(baseURL, account_id)
    canvas_list = get_items(url)
    dsc_df = pd.io.json.json_normalize(canvas_list)
    
    # Transitional sub account (835)
    print_log('Trans 835')
    account_id = 835     
    url='{}accounts/{}/courses?per_page=100'.format(baseURL, account_id)
    canvas_list = get_items(url)
    trans_df = pd.io.json.json_normalize(canvas_list)
    
    # Combine and save excel file
    print_log('Combine  ')
    canvas_df = pd.concat([trans_df, dsc_df])
    canvas_df['url']= "https://rmit.instructure.com/courses/" + canvas_df['id'].astype(str)
    canvas_df = canvas_df[['sis_course_id','course_code','account_id','id','workflow_state','url','default_view']]
    canvas_df.to_excel('./report/Canvas-URLs_{}.xlsx'.format(printDate))
    
    print_log('End      ')


## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## API call to get Canvas items
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def get_items(url):
    items =[]
    items_json = []
    
    r = requests.get(url, headers = header)   
    if r.status_code == requests.codes.ok:
        items_json = r.json()
    
    for item in items_json:  
        items.append(item)
      
    while r.links['current']['url'] != r.links['last']['url']:
            r = requests.get(r.links['next']['url'], headers=header)
            items_json = r.json()
            for item in items_json:
                items.append(item)

    return items

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## Write json data to an excel file
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def write_to_excel(data, tab, file):
    data_df = pd.io.json.json_normalize(data)
    try:
        writer = pd.ExcelWriter(file)
        data_df.to_excel(writer, sheet_name=tab)
        writer.save()
    except:
        # need errorhandler - file open
        print("Error writing to file:")

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## Write message to the log file
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def print_log(msg):    
    log_time = str(time.asctime(time.localtime(time.time())))   
    message = "{}: {}".format(msg, log_time)
    write_to_log(message) 
    
def write_to_log(message):
    with open(log_file, 'a') as log:               
        log.write(message + "\n")     # write to file              
        pprint(message)               # print to screen
        
    
    
if __name__ == "__main__": main()

'DSC   700: Wed May 16 00:20:04 2018'
'Trans 835: Wed May 16 00:32:33 2018'
'Combine  : Wed May 16 00:41:25 2018'
'End      : Wed May 16 00:41:30 2018'
